# Amenity Radar Chart

Spider/radar chart comparing amenity categories for selected neighbourhoods.

## 1. Data Reference

### Source Tables

| Table | Grain | Key Columns |
|-------|-------|-------------|
| `mart_neighbourhood_amenities` | neighbourhood × year | parks_index, schools_index, transit_index |

### SQL Query

In [1]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load .env from project root
load_dotenv("../../.env")

engine = create_engine(os.environ["DATABASE_URL"])

query = """
SELECT
    neighbourhood_name,
    parks_index,
    schools_index,
    transit_index,
    amenity_index,
    amenity_tier
FROM mart_toronto.mart_neighbourhood_amenities
WHERE year = (SELECT MAX(year) FROM mart_toronto.mart_neighbourhood_amenities)
  AND amenity_index IS NOT NULL
ORDER BY amenity_index DESC
"""

df = pd.read_sql(query, engine)
print(f"Loaded {len(df)} neighbourhoods")

Loaded 158 neighbourhoods


### Transformation Steps

1. Select top 5 and bottom 5 neighbourhoods by amenity index
2. Reshape for radar chart format

In [2]:
# Select representative neighbourhoods
top_5 = df.head(5)
bottom_5 = df.tail(5)

# Prepare radar data
categories = ["Parks", "Schools", "Transit"]
index_columns = ["parks_index", "schools_index", "transit_index"]

### Sample Output

In [3]:
print("Top 5 Amenity-Rich Neighbourhoods:")
display(
    top_5[
        [
            "neighbourhood_name",
            "parks_index",
            "schools_index",
            "transit_index",
            "amenity_index",
        ]
    ]
)
print("\nBottom 5 Underserved Neighbourhoods:")
display(
    bottom_5[
        [
            "neighbourhood_name",
            "parks_index",
            "schools_index",
            "transit_index",
            "amenity_index",
        ]
    ]
)

Top 5 Amenity-Rich Neighbourhoods:


,neighbourhood_name,parks_index,schools_index,transit_index,amenity_index
0,Bridle Path-Sunnybrook-York Mills,279.9,284.5,246.3,253.8
1,West Humber-Clairville,102.1,152.9,288.0,234.3
2,Humber Summit,146.7,75.9,274.8,222.8
3,Yorkdale-Glen Park,150.5,167.0,237.7,211.7
4,St.Andrew-Windfields,189.6,218.1,211.6,200.1



Bottom 5 Underserved Neighbourhoods:


,neighbourhood_name,parks_index,schools_index,transit_index,amenity_index
153,Harbourfront-CityPlace,101.7,32.8,16.1,30.9
154,North Toronto,33.8,102.0,20.9,30.7
155,North St.James Town,29.0,37.5,24.4,27.5
156,Avondale,64.9,50.4,17.5,26.1
157,Yonge-Doris,53.3,45.8,17.9,25.2


## 2. Data Visualization

### Figure Factory

Uses `create_radar` from `portfolio_app.figures.toronto.radar`.

In [4]:
import sys

sys.path.insert(0, "../..")

from portfolio_app.figures.toronto.radar import create_comparison_radar

# Compare top neighbourhood vs city average (100)
top_hood = top_5.iloc[0]
metrics = ["parks_index", "schools_index", "transit_index"]

fig = create_comparison_radar(
    selected_data=top_hood.to_dict(),
    average_data={"parks_index": 100, "schools_index": 100, "transit_index": 100},
    metrics=metrics,
    selected_name=top_hood["neighbourhood_name"],
    average_name="City Average",
    title=f"Amenity Profile: {top_hood['neighbourhood_name']} vs City Average",
)

fig.show()

### Index Interpretation

| Value | Meaning |
|-------|--------|
| < 100 | Below city average |
| = 100 | City average |
| > 100 | Above city average |